In [23]:
import isaid_helpers
import pandas as pd

In [24]:
pd.read_csv(isaid_helpers.f_graphable_doi).head()

,doi,name,url,publisher,date_qualifier,year_published,entity_type,journal,description,event
0,10.3133/ofr2000247,Fish community structure in relation to enviro...,http://dx.doi.org/10.3133/OFR2000247,US Geological Survey,2021-03-31T06:34:09.293702,2000.0,CreativeWork,USGS Open-File Report,NaN,NaN
1,10.1002/eap.2243,Resistance and resilience of pelagic and litto...,http://dx.doi.org/10.1002/EAP.2243,Wiley,2021-03-31T06:37:06.414581,2021.0,CreativeWork,Ecological Applications,NaN,NaN
2,10.3133/sir20175079,"Groundwater discharge by evapotranspiration, f...",http://dx.doi.org/10.3133/SIR20175079,US Geological Survey,2021-04-01T07:31:15.102431,2017.0,CreativeWork,USGS Scientific Investigations Report,NaN,NaN
3,10.3133/ofr92484,Selected meteorological data for an arid site ...,http://dx.doi.org/10.3133/OFR92484,US Geological Survey,2021-04-01T07:31:17.221993,1992.0,CreativeWork,USGS Open-File Report,NaN,NaN
4,10.3133/ofr20071239,Visitor and community survey results for Prime...,http://dx.doi.org/10.3133/OFR20071239,US Geological Survey,2021-04-01T00:30:40.548623,2007.0,CreativeWork,USGS Open-File Report,NaN,NaN


In [25]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE row.entity_type = "CreativeWork"
                MERGE (cw:CreativeWork {doi: row.doi})
                ON CREATE
                    SET cw.name = row.name,
                    cw.url = row.url,
                    cw.publisher = row.publisher,
                    cw.year_published = row.year_published,
                    cw.journal = row.journal,
                    cw.description = row.description,
                    cw.event = row.event
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 2.16 ms, sys: 6.26 ms, total: 8.42 ms
Wall time: 225 ms


In [26]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE row.entity_type = "Dataset"
                MERGE (cw:Dataset {doi: row.doi})
                ON CREATE
                    SET cw.name = row.name,
                    cw.url = row.url,
                    cw.publisher = row.publisher,
                    cw.year_published = row.year_published,
                    cw.journal = row.journal,
                    cw.description = row.description,
                    cw.event = row.event
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.01 ms, sys: 1.24 ms, total: 2.25 ms
Wall time: 4.82 s


In [27]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE NOT row.event IS NULL
                MERGE (e:Event {name: row.event})
                ON CREATE
                    SET e.original_source = "DOI Record",
                    e.reference = row.url
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.48 ms, sys: 2.06 ms, total: 3.54 ms
Wall time: 177 ms


In [28]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE NOT row.journal IS NULL
                MERGE (j:Journal {name: row.journal})
                ON CREATE
                    SET j.original_source = "DOI Record"
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.36 ms, sys: 2.45 ms, total: 3.82 ms
Wall time: 12.7 s


In [29]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE NOT row.event IS NULL
                MATCH (cw:CreativeWork {doi: row.doi})
                WITH cw, row
                    MATCH (e:Event {name: row.event})
                    MERGE (cw)-[cwe:PRODUCT_OF]->(e)
                        SET cwe.reference = row.url,
                        cwe.date_qualifier = row.year_published
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 992 µs, sys: 1.22 ms, total: 2.21 ms
Wall time: 538 ms


In [30]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE NOT row.journal IS NULL
                MATCH (cw:CreativeWork {doi: row.doi})
                WITH cw, row
                    MATCH (j:Journal {name: row.journal})
                    MERGE (cw)-[cwj:PUBLISHED_IN]->(j)
                        SET cwj.reference = row.url,
                        cwj.date_qualifier = row.year_published
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.3 ms, sys: 1.39 ms, total: 2.69 ms
Wall time: 13.9 s


In [31]:
pd.read_csv(isaid_helpers.f_graphable_doi_contacts).head()

,doi,reference,date_qualifier,orcid,sequence,rel_type,entity_type,name
0,10.3133/ofr2000247,http://dx.doi.org/10.3133/OFR2000247,2000,0000-0001-6702-4531,additional,AUTHOR_OF,Person,Larry R. Brown
1,10.1002/eap.2243,http://dx.doi.org/10.1002/EAP.2243,2021,0000-0001-6702-4531,additional,AUTHOR_OF,Person,Larry Brown
2,10.3133/sir20175079,http://dx.doi.org/10.3133/SIR20175079,2017,0000-0002-2086-0417,additional,AUTHOR_OF,Person,Brian J. Andraski
3,10.3133/sir20175079,http://dx.doi.org/10.3133/SIR20175079,2017,0000-0003-3776-3565,additional,AUTHOR_OF,Person,C. Amanda Garcia
4,10.3133/ofr92484,http://dx.doi.org/10.3133/OFR92484,1992,0000-0002-2086-0417,additional,AUTHOR_OF,Person,Brian J. Andraski


In [32]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE row.rel_type = "AUTHOR_OF"
                MATCH (cw:CreativeWork {doi: row.doi})
                WITH cw, row
                    MATCH (p:Person {orcid: row.orcid})
                    MERGE (p)-[a:AUTHOR_OF]->(cw)
                    ON CREATE
                        SET a.reference = row.reference,
                        a.date_qualifier = row.date_qualifier,
                        a.sequence = row.sequence
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi_contacts
    })

CPU times: user 3.83 ms, sys: 2.43 ms, total: 6.26 ms
Wall time: 2min 54s


In [16]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE row.rel_type = "EDITOR_OF"
                MATCH (cw:CreativeWork {doi: row.doi})
                WITH cw, row
                    MATCH (p:Person {orcid: row.orcid})
                    MERGE (p)-[a:EDITOR_OF]->(cw)
                    ON CREATE
                        SET a.reference = row.reference,
                        a.date_qualifier = row.date_qualifier,
                        a.sequence = row.sequence
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi_contacts
    })

CPU times: user 1.15 ms, sys: 2.77 ms, total: 3.92 ms
Wall time: 1.82 s


In [18]:
pd.read_csv(isaid_helpers.f_graphable_doi_funders).head()

,doi,reference,date_qualifier,name,rel_type,entity_type,funder_doi,funder_award
0,10.1002/eap.2243,http://dx.doi.org/10.1002/EAP.2243,2021,Department of Water Resources,FUNDER_OF,Organization,10.13039/100004813,NaN
1,10.1002/eap.2243,http://dx.doi.org/10.1002/EAP.2243,2021,Bureau of Reclamation,FUNDER_OF,Organization,10.13039/100006450,NaN
2,10.1016/j.jglr.2013.12.011,http://dx.doi.org/10.1016/J.JGLR.2013.12.011,2014,U.S. Environmental Protection Agency,FUNDER_OF,Organization,10.13039/100000139,NaN
3,10.1080/10871209.2017.1324069,http://dx.doi.org/10.1080/10871209.2017.1324069,2017,"South Dakota Game, Fish and Parks",FUNDER_OF,Organization,10.13039/100014298,T-59-R1
4,10.1016/j.biocon.2016.05.015,http://dx.doi.org/10.1016/J.BIOCON.2016.05.015,2017,U.S. Forest Service and the Ecological Society...,FUNDER_OF,Organization,NaN,12-CA-11221633-096


In [19]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            MERGE (o:Organization {name: row.name})
            ON CREATE
                SET o.category = "Funding Organization",
                o.original_source = "DOI Record",
                o.doi = row.funder_doi

                WITH o, row
                    MATCH (cw:CreativeWork {doi: row.doi})
                    MERGE (cw)-[fb:FUNDED_BY]->(o)
                    ON CREATE
                        SET fb.reference = row.reference,
                        fb.date_qualifier = row.date_qualifier,
                        fb.funder_award = row.funder_award
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi_contacts
    })

CPU times: user 1.21 ms, sys: 2.54 ms, total: 3.74 ms
Wall time: 681 ms


In [33]:
pd.read_csv(isaid_helpers.f_graphable_doi_terms).head()

,doi,reference,date_qualifier,name,rel_type,entity_type
0,10.1002/eap.2243,http://dx.doi.org/10.1002/EAP.2243,2021,Ecology,ADDRESSES_SUBJECT,UndefinedSubjectMatter
1,10.1016/j.jglr.2013.12.011,http://dx.doi.org/10.1016/J.JGLR.2013.12.011,2014,Ecology,ADDRESSES_SUBJECT,UndefinedSubjectMatter
2,10.1016/j.jglr.2013.12.011,http://dx.doi.org/10.1016/J.JGLR.2013.12.011,2014,Aquatic Science,ADDRESSES_SUBJECT,UndefinedSubjectMatter
3,10.1016/j.jglr.2013.12.011,http://dx.doi.org/10.1016/J.JGLR.2013.12.011,2014,"Ecology, Evolution, Behavior and Systematics",ADDRESSES_SUBJECT,UndefinedSubjectMatter
4,10.1111/conl.12095,http://dx.doi.org/10.1111/CONL.12095,2014,Ecology,ADDRESSES_SUBJECT,UndefinedSubjectMatter


In [34]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            MERGE (t:UndefinedSubjectMatter {name: row.name})
            ON CREATE
                SET t.original_source = "DOI Record"

                WITH t, row
                    MATCH (cw:CreativeWork {doi: row.doi})
                    MERGE (cw)-[s:ADDRESSES_SUBJECT]->(t)
                    ON CREATE
                        SET s.reference = row.reference,
                        s.date_qualifier = row.date_qualifier
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi_terms
    })

CPU times: user 1.46 ms, sys: 4.93 ms, total: 6.39 ms
Wall time: 299 ms
